In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from sklearn.metrics import mean_squared_error, precision_score, recall_score, ndcg_score, average_precision_score

# Importing matplotlib.pyplot
import matplotlib.pyplot as plt # This line imports the necessary module
import seaborn as sns # Importing seaborn for heatmap visualization

# 1. 데이터 전처리
# 파일 경로 정의
movies_path = '/content/drive/MyDrive/ml-latest-small/movies.csv'
tags_path = '/content/drive/MyDrive/ml-latest-small/tags.csv'

# 데이터 불러오기
movies_df = pd.read_csv(movies_path)
tags_df = pd.read_csv(tags_path)

# movieId와 title 칼럼만 확인
print(f"Movie data shape: {movies_df.shape}")
print(movies_df.head())

# 영화 제목의 유니크한 개수 확인
print(f"Unique movie titles: {movies_df['title'].nunique()}")

# 영화 제목의 중복 여부 확인
duplicates = movies_df[movies_df.duplicated(subset='title', keep=False)]
print(f"Number of duplicate movie titles: {duplicates.shape[0]}")
print(duplicates[['movieId', 'title']].sort_values(by='title'))

# 중복된 영화 제목 제거 (첫 번째 항목만 남기고 나머지는 삭제)
movies_df_clean = movies_df.drop_duplicates(subset='title', keep='first')
print(f"Cleaned movie data shape: {movies_df_clean.shape}")

# 중복된 제목이 없는지 확인
duplicate_titles_clean = movies_df_clean[movies_df_clean.duplicated(subset='title', keep=False)]
print(f"Number of duplicate movie titles after cleaning: {duplicate_titles_clean.shape[0]}")

Movie data shape: (9742, 3)
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
Unique movie titles: 9737
Number of duplicate movie titles: 10
      movieId                                   title
4169     6003  Confessions of a Dangerous Mind (2002)
9106   144606  Confessions of a Dangerous Mind (2002)
650       838                             Emma (1996)
5601    26958                             Emma (1996)
5854    32600                             Er

**Jaccard Similarity Matrix**

In [ ]:
from tqdm.notebook import tqdm

def jaccard_similarity(set1, set2):
    """
    두 집합 간의 Jaccard 유사도를 계산합니다.

    Args:
        set1 (set): 첫 번째 집합
        set2 (set): 두 번째 집합

    Returns:
        float: Jaccard 유사도
    """
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union else 0

def create_similarity_matrix(movie_info):
    """
    영화 정보 데이터프레임을 입력으로 받아 영화 간의 유사도 매트릭스를 생성합니다.

    Args:
        movie_info (pd.DataFrame): 영화 정보 데이터프레임

    Returns:
        pd.DataFrame: 영화 간의 유사도 매트릭스
    """
    # 영화 ID 리스트
    movie_ids = movie_info['movieId'].tolist()
    n_movies = len(movie_ids)

    # 유사도 매트릭스 초기화 (DataFrame)
    similarity_matrix = pd.DataFrame(index=movie_ids, columns=movie_ids)

    # 유사도 계산 및 매트릭스 채우기
    for i in tqdm(range(n_movies), desc="Calculating similarity matrix"):
        for j in range(i, n_movies):  # i에서 시작하여 자기 자신 포함
            movie_id1 = movie_ids[i]
            movie_id2 = movie_ids[j]

            # Jaccard Similarity 계산
            tags1 = movie_info.loc[movie_info['movieId'] == movie_id1, 'tags'].values[0]
            tags2 = movie_info.loc[movie_info['movieId'] == movie_id2, 'tags'].values[0]

            similarity = jaccard_similarity(tags1, tags2)

            # 유사도 매트릭스에 값 저장
            similarity_matrix.loc[movie_id1, movie_id2] = similarity
            similarity_matrix.loc[movie_id2, movie_id1] = similarity  # 대칭 행렬

    # 대각선 요소는 1로 설정 (자기 자신과의 유사도)
    for movie_id in movie_ids:
        similarity_matrix.loc[movie_id, movie_id] = 1

    return similarity_matrix

# 1. movies_df_clean과 tags_df를 병합 (모든 항목 유지)
#    - how='left'로 병합하여 태그가 없는 영화도 유지
merged_df = pd.merge(movies_df_clean, tags_df, on='movieId', how='left')

# 병합 후 영화 개수 출력
num_movies_after_merge = merged_df['movieId'].nunique()
print(f"병합 후 영화 개수: {num_movies_after_merge}")

# 2. Genres 데이터를 리스트로 변환
merged_df['genres_list'] = merged_df['genres'].apply(lambda x: x.split('|'))

# 3. Tags를 소문자로 변환 및 공백 제거
merged_df['tag'] = merged_df['tag'].str.lower().str.strip()

# 4. 각 영화에 대한 고유 태그 및 장르 집합 생성
movie_info = merged_df.groupby('movieId').agg({
    'tag': lambda x: set(x.dropna()),  # NaN 제거 후 set으로 변환
    'genres_list': lambda x: set(sum(x, []))  # 중첩 리스트를 하나의 set으로 변환
}).reset_index()

# 컬럼 이름 변경 (태그와 장르 구분 명확히)
movie_info.rename(columns={'tag': 'tags', 'genres_list': 'genres'}, inplace=True)

# 5. 영화-영화 매트릭스 생성 (NumPy 사용 안 함)
combined_similarity_matrix = create_similarity_matrix(movie_info)


병합 후 영화 개수: 9737


Calculating similarity matrix:   0%|          | 0/9737 [00:00<?, ?it/s]

**Skip-gram**

In [ ]:
from gensim.models import Word2Vec

# 각 영화에 대해 상위 K개의 가장 가까운 영화 찾기
def generate_sequences(pmi_matrix, movie_ids, k=5):
    """
    PMI 매트릭스를 기반으로 각 영화에 대해 상위 K개의 가장 가까운 영화를 찾습니다.

    Args:
        pmi_matrix (np.ndarray): 영화 간 PMI 유사도 매트릭스
        movie_ids (list): 영화 ID 리스트
        k (int): 선택할 유사한 영화 개수

    Returns:
        list: (영화 ID, 유사한 영화 ID)의 시퀀스 리스트
    """
    sequences = []

    for i, movie_id in enumerate(movie_ids):
        # PMI 점수를 기준으로 가장 가까운 영화 선택
        similar_indices = np.argsort(pmi_matrix[i])[::-1]  # 유사도 내림차순 정렬

        for idx in similar_indices[:k]:
            if i != idx and pmi_matrix[i, idx] > 0:  # 자기 자신 제외, PMI가 0보다 큰 경우만 추가
                sequences.append((movie_id, movie_ids[idx]))

    return sequences

# Skip-gram 시퀀스 생성
movie_ids = list(range(combined_similarity_matrix.shape[0]))  # 영화 ID 리스트
skip_gram_sequences = generate_sequences(combined_similarity_matrix, movie_ids, k=5)
print(f"Generated {len(skip_gram_sequences)} skip-gram sequences.")

# 시퀀스를 Word2Vec 입력 형식으로 변환
skip_gram_pairs = [[str(pair[0]), str(pair[1])] for pair in skip_gram_sequences]

# Word2Vec 모델 학습
model = Word2Vec(
    sentences=skip_gram_pairs,  # 학습 데이터 (시퀀스)
    vector_size=100,  # 벡터 차원 수
    window=5,  # 학습 윈도우 크기
    sg=1,  # Skip-gram 사용 (1)
    workers=4,  # 병렬 처리 워커 수
    min_count=1  # 최소 등장 횟수 필터링
)
print("Word2Vec model trained.")

# 특정 영화의 임베딩 확인
example_movie_id = movie_ids[0]  # 예시로 첫 번째 영화 선택
embedding = model.wv[str(example_movie_id)]
print(f"Embedding for movie ID {example_movie_id}: {embedding}")


In [ ]:
def find_movies_by_tag(input_tag, movie_info, combined_similarity_matrix, movie_ids, movies_df, top_k=5):
    """
    특정 태그와 유사한 영화를 찾는 함수.
    Args:
        input_tag (str): 검색할 태그
        movie_info (pd.DataFrame): 영화 정보 데이터프레임
        combined_similarity_matrix (numpy.ndarray): 유사도 매트릭스
        movie_ids (list): 영화 ID 리스트
        movies_df (pd.DataFrame): 영화 제목 포함 데이터프레임
        top_k (int): 상위 몇 개의 영화를 출력할지 지정
    Returns:
        list: 관련 영화 제목 및 Jaccard 유사도 점수
    """
    input_tag = input_tag.lower().strip()  # 태그를 소문자로 정리
    tag_similarities = []

    for i, movie_id in enumerate(movie_ids):
        movie_tags = movie_info.loc[movie_info['movieId'] == movie_id, 'tag'].values[0]

        if isinstance(movie_tags, (list, set)):  # 태그가 유효할 경우
            jaccard_score = jaccard_similarity(set([input_tag]), set(movie_tags))
            tag_similarities.append((movie_id, jaccard_score))

    tag_similarities = sorted(tag_similarities, key=lambda x: x[1], reverse=True)[:top_k]

    result = []
    for movie_id, score in tag_similarities:
        if score > 0:  # 유사도가 0 이상인 경우만
            title = movies_df.loc[movies_df['movieId'] == movie_id, 'title'].values[0]
            result.append((title, score))

    return result


In [ ]:
def find_most_similar_movies(movie_id, similarity_matrix, movie_ids, movies_df, top_k=5):
    """
    특정 영화와 가장 유사한 영화를 찾는 함수.
    Args:
        movie_id (int): 기준이 되는 영화 ID
        similarity_matrix (numpy.ndarray): 유사도 매트릭스
        movie_ids (list): 영화 ID 리스트
        movies_df (pd.DataFrame): 영화 제목 포함 데이터프레임
        top_k (int): 상위 몇 개의 유사 영화를 출력할지 지정
    Returns:
        list: 유사한 영화 제목 및 유사도 점수
    """
    movie_index = movie_ids.index(movie_id)
    similarities = similarity_matrix[movie_index]

    # 자기 자신 제외하고 유사도 높은 순으로 정렬
    similar_indices = np.argsort(similarities)[::-1]
    similar_indices = [idx for idx in similar_indices if idx != movie_index][:top_k]

    similar_movies = []
    for idx in similar_indices:
        similar_movie_id = movie_ids[idx]
        similar_title = movies_df.loc[movies_df['movieId'] == similar_movie_id, 'title'].values[0]
        similar_score = similarities[idx]
        similar_movies.append((similar_title, similar_score))

    return similar_movies


In [ ]:
# 예제: 특정 태그로 영화 추천
input_tag = "adventure"
related_movies = find_movies_by_tag(input_tag, movie_info, combined_similarity_matrix, movie_ids, movies_df_clean, top_k=5)

print(f"Movies related to the tag '{input_tag}':")
for title, score in related_movies:
    print(f" - {title}: Jaccard Similarity = {score:.4f}")

# 예제: 특정 영화와 유사한 영화 추천
example_movie_id = movie_ids[0]  # 예시로 첫 번째 영화 ID 사용
similar_movies = find_most_similar_movies(example_movie_id, combined_similarity_matrix, movie_ids, movies_df_clean, top_k=5)

print(f"Movies most similar to '{movies_df_clean.loc[movies_df_clean['movieId'] == example_movie_id, 'title'].values[0]}':")
for title, score in similar_movies:
    print(f" - {title}: Similarity Score = {score:.4f}")


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def find_similar_movies_analogy(movie_id, similarity_matrix, movie_ids, movies_df, top_k=5):
    """
    주어진 영화 ID에 대해 가장 유사한 영화를 찾는 함수 (tag, genre 유사도 기반).

    Args:
        movie_id (int): 기준 영화 ID
        similarity_matrix (numpy.ndarray): tag, genre 유사도 매트릭스
        movie_ids (list): 영화 ID 리스트
        movies_df (pd.DataFrame): 영화 제목 포함 데이터프레임
        top_k (int): 상위 K개의 영화 반환

    Returns:
        list: 가장 유사한 영화 목록 (제목 및 유사도)
    """
    # 1. 해당 영화 ID에 대한 인덱스를 가져오기
    movie_index = movie_ids.index(movie_id)

    # 2. 해당 영화와 다른 영화들 간의 유사도 추출
    similarities = similarity_matrix[movie_index]

    # 3. 유사도 기준 내림차순 정렬 (자기 자신 제외)
    similar_indices = np.argsort(similarities)[::-1]
    similar_indices = [idx for idx in similar_indices if idx != movie_index][:top_k]

    # 4. 유사한 영화 ID, 제목 및 유사도 리스트 생성
    similar_movies = []
    for idx in similar_indices:
        similar_movie_id = movie_ids[idx]
        similar_title = movies_df[movies_df['movieId'] == similar_movie_id]['title'].values[0]
        similar_score = similarities[idx]
        similar_movies.append((similar_title, similar_score))

    return similar_movies

# 예시: 특정 영화 ID로 유사한 영화 찾기
example_movie_id = movie_ids[0]  # 데이터셋의 첫 번째 영화 선택

similar_movies = find_similar_movies_analogy(
    example_movie_id, combined_similarity_matrix, movie_ids, movies_df_clean, top_k=5
)

# 결과 출력
print(
    f"'{movies_df_clean[movies_df_clean['movieId'] == example_movie_id]['title'].values[0]}' 영화와 유사한 영화:"
)
for title, score in similar_movies:
    print(f" - {title}: 유사도 = {score:.4f}")


def plot_movie_embeddings(model, selected_movies, movie_titles, perplexity=30, n_iter=300):
    """
    영화 임베딩을 t-SNE로 시각화하는 함수.

    Args:
        model (Word2Vec): Word2Vec 모델
        selected_movies (list): 강조할 영화 ID 리스트
        movie_titles (dict): 영화 ID와 제목 매핑 딕셔너리
        perplexity (int): t-SNE의 perplexity 값
        n_iter (int): t-SNE의 반복 횟수
    """
    # 1. Word2Vec 모델에서 모든 영화의 벡터 추출
    movie_ids = list(model.wv.index_to_key)
    embeddings = np.array([model.wv[movie_id] for movie_id in movie_ids])

    # 2. t-SNE를 사용하여 차원 축소
    tsne = TSNE(n_components=2, perplexity=perplexity, n_iter=n_iter, random_state=42)
    tsne_result = tsne.fit_transform(embeddings)

    # 3. 시각화
    plt.figure(figsize=(10, 8))
    plt.scatter(tsne_result[:, 0], tsne_result[:, 1], s=5, alpha=0.5, label="Movies")

    # 4. 특정 영화 강조 (어휘에 존재하는 경우)
    for movie_id in selected_movies:
        if movie_id in model.wv and movie_id in movie_titles:
            idx = movie_ids.index(movie_id)
            x, y = tsne_result[idx]
            plt.scatter(x, y, s=50, label=movie_titles[movie_id])
            plt.annotate(movie_titles[movie_id], (x, y), fontsize=9, color="red")

    plt.title("2D Visualization of Movie Embeddings using t-SNE")
    plt.xlabel("t-SNE Dimension 1")
    plt.ylabel("t-SNE Dimension 2")
    plt.legend()
    plt.show()


# Word2Vec 모델에서 영화 ID와 제목 매핑
selected_movie_ids = ['1', '122918', '2355', '108932']
movie_titles = {
    '1': 'Guardians of the Galaxy 2 (2017)',
    '122918': 'Picnic at Hanging Rock (1975)',
    '2355': 'Bug\'s Life, A (1998)',
    '108932': 'The Lego Movie (2014)'
}

# Word2Vec 모델 어휘에 있는지 확인
for movie_id in selected_movie_ids:
    if movie_id in model.wv:
        print(f"Movie ID '{movie_id}' is in the vocabulary.")
    else:
        print(f"Movie ID '{movie_id}' is NOT in the vocabulary.")

# t-SNE 시각화 호출
plot_movie_embeddings(model, selected_movie_ids, movie_titles)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


# 코사인 유사도로 유사한 영화 찾는 함수
def find_similar_movies_cosine(movie_id, movie_vectors, movie_ids, movies_df, top_k=5):
    """
    주어진 영화 ID에 대해 코사인 유사도를 사용하여 가장 유사한 영화를 찾는 함수.
    """
    try:
        movie_index = movie_ids.index(movie_id)  # movie_ids가 문자열로 저장되어 있다고 가정
        similarities = cosine_similarity(movie_vectors[movie_index].reshape(1, -1), movie_vectors).flatten()
        similar_indices = np.argsort(similarities)[::-1]
        similar_indices = [idx for idx in similar_indices if idx != movie_index][:top_k]
        similar_movies = [
            (
                movies_df[movies_df['movieId'] == int(movie_ids[idx])]['title'].values[0],
                similarities[idx]
            )
            for idx in similar_indices
        ]
        return similar_movies
    except ValueError:
        print(f"Movie ID '{movie_id}' not found in movie_ids.")
        return []

# t-SNE 기반 영화 임베딩 시각화 함수
def plot_movie_embeddings_cosine(movie_vectors, movie_ids, selected_movie_ids, movie_titles, perplexity=30, n_iter=300):
    """
    영화 임베딩을 t-SNE로 시각화하는 함수 (코사인 유사도 기반).
    """
    tsne = TSNE(n_components=2, perplexity=perplexity, n_iter=n_iter, random_state=42)
    tsne_result = tsne.fit_transform(movie_vectors)

    plt.figure(figsize=(10, 8))
    plt.scatter(tsne_result[:, 0], tsne_result[:, 1], s=5, alpha=0.5, label="Movies")

    for movie_id in selected_movie_ids:
        if movie_id in movie_ids:
            idx = movie_ids.index(movie_id)
            x, y = tsne_result[idx]
            movie_title = movie_titles.get(movie_id, "Unknown")
            plt.scatter(x, y, s=50, label=movie_title)
            plt.annotate(movie_title, (x, y), fontsize=9, color="red")

    plt.title("2D Visualization of Movie Embeddings using t-SNE (Cosine Similarity)")
    plt.xlabel("t-SNE Dimension 1")
    plt.ylabel("t-SNE Dimension 2")
    plt.legend()
    plt.show()

# 영화 벡터 생성 (테스트 데이터)
movie_vectors = np.random.rand(len(movies_df_clean), 10)  # 10차원 랜덤 벡터
example_movie_id = movie_ids[1]  # 테스트용 ID
similar_movies = find_similar_movies_cosine(
    example_movie_id, movie_vectors, movie_ids, movies_df_clean, top_k=5
)

print(f"'{movies_df_clean[movies_df_clean['movieId'] == int(example_movie_id)]['title'].values[0]}' 영화와 유사한 영화:")
for title, score in similar_movies:
    print(f" - {title}: 유사도 = {score:.4f}")

# 선택 영화 ID와 제목
selected_movie_ids = ['2', '96606', '3117', '3922']
movie_titles = {
    '2': 'Jumanji (1995)',
    '96606': 'Samsara (2011)',
    '3117': 'Ride with the Devil (1999)',
    '3922': 'Bikini Beach (1964)'
}

# 선택된 영화 확인 및 시각화
for movie_id in selected_movie_ids:
    print(f"Movie ID '{movie_id}' in movie_ids: {'YES' if movie_id in movie_ids else 'NO'}")
    print(f"Movie ID '{movie_id}' in movie_titles: {'YES' if movie_id in movie_titles else 'NO'}")

plot_movie_embeddings_cosine(movie_vectors, movie_ids, selected_movie_ids, movie_titles)
